In [1]:
from __future__ import division


import tensorflow as tf
import os
import pandas as pd
import numpy as np

from nets import inception_utils, inception_v3


/Users/gaoqitong/anaconda/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
base_skin_dir = os.path.join('./Data/')
slim = tf.contrib.slim

tile_df = pd.read_csv(os.path.join(base_skin_dir, 'hmnist_28_28_L.csv'))
input_images = np.reshape(tile_df.values[:,:-1],[-1,28,28,1]).astype(np.float32)
input_labels = tile_df.values[:,-1]
input_size = np.shape(input_images)[0]

images = tf.constant(input_images[:9984], dtype=tf.float32)
labels = tf.constant(input_labels[:9984])
labels = tf.one_hot(labels, depth=7)

In [3]:
def get_checkpoint_init_fn():
    variables_to_restore = slim.get_variables_to_restore()
    global_step_reset = tf.assign(tf.train.get_or_create_global_step(), 0)
    slim_init_fn = slim.assign_from_checkpoint_fn("./inception-v3/model.ckpt-157585",variables_to_restore,ignore_missing_vars=True)

In [4]:
g = tf.Graph()
with g.as_default():
    dataset = tf.data.Dataset.from_tensor_slices((input_images,input_labels)).batch(64)
    iterator = dataset.make_one_shot_iterator()
    image, label = iterator.get_next()
    label = tf.one_hot(label, depth=7)
    logits, end_points = inception_v3.inception_v3(image, 7, "Mixed_5d", create_aux_logits=False)
    loss = tf.losses.softmax_cross_entropy(label, logits)
    learning_rate = tf.train.exponential_decay(1e-04, tf.train.get_or_create_global_step(), input_size / 64 * 2.5 , 0.94)
    opt = tf.train.AdamOptimizer(learning_rate)
    train_tensor = slim.learning.create_train_op(loss, optimizer=opt)
    slim.summaries.add_scalar_summary(loss, 'cross_entropy_loss', 'losses')
    slim.summaries.add_scalar_summary(learning_rate, 'learning_rate', 'training')
    slim.learning.train(
        train_tensor,
        "./saved_model/",
        log_every_n_steps=300,
        graph=g,
        save_summaries_secs=300,
        save_interval_secs=600,
        init_fn=get_checkpoint_init_fn(),
        global_step=tf.train.get_global_step())

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Restoring parameters from ./saved_model/model.ckpt-468
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path ./saved_model/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:Recording summary at step 468.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global step 600: loss = 0.0001 (0.032 sec/step)
INFO:tensorflow:Caught OutOfRangeError. Stopping Training. End of sequence
	 [[node IteratorGetNext (defined at <ipython-input-4-59f245f0e3fa>:5)  = IteratorGetNext[output_shapes=[[?,28,28,1], [?]], output_types=[DT_FLOAT, DT_INT64], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator)]]

Caused by op u'IteratorGetNext', defined at:
  File "/Users/gaoqitong/anaconda/envs/py27/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, 

INFO:tensorflow:Finished training! Saving model to disk.
